In [19]:
import pandas as pd
from surprise import Reader, Dataset, SVD, evaluate
reader = Reader()

ratings = pd.read_csv('item_rating.csv')
ratings.head()

,userId,gender,itemId,rating
0,1,2,16,3
1,1,2,11,1
2,1,2,8,1
3,1,2,15,1
4,1,2,19,5


In [2]:
data = Dataset.load_from_df(ratings[['userId', 'itemId', 'rating']], reader)
# 교차검증을 위한 데이터분할 
data.split(n_folds=5)

In [3]:
import random
import warnings
warnings.filterwarnings(action='ignore')

svd = SVD()
# 평균 제곱근 편차, 평균 절대 오차 계산 
evaluate(svd, data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 1.4781
MAE:  1.2804
------------
Fold 2
RMSE: 1.4437
MAE:  1.2407
------------
Fold 3
RMSE: 1.3307
MAE:  1.1453
------------
Fold 4
RMSE: 1.3593
MAE:  1.2129
------------
Fold 5
RMSE: 1.4535
MAE:  1.2388
------------
------------
Mean RMSE: 1.4131
Mean MAE : 1.2236
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [1.4781424460119945,
                             1.4436996842439875,
                             1.3307060295825501,
                             1.3592976738692324,
                             1.453536937941804],
                            'mae': [1.2803811401112741,
                             1.2406507383015437,
                             1.1453026602262741,
                             1.2129151854295357,
                             1.2387509320232017]})

In [4]:
trainset = data.build_full_trainset()
# 데이터셋 학습시키기 
svd.fit(trainset)

In [5]:
svd.predict(1,7)

Prediction(uid=1, iid=7, r_ui=None, est=2.7817485360359617, details={'was_impossible': False})

In [6]:
'''
#itemid 
1 = shorts
2 = caps
3 = T-shirt
4 = sandals
5 = sweater
6 = cardigan
7 = raincoat
8 = coat
9 = gloves
10 = jeans
11 = sneakers
12 = dress
13 = suit
14 = pants
15 = vest
16 = blouse
17 = jacket
18 = sunglasses
19 = bag
20 = shoes
'''
item_keyword = ['shorts','caps','T-shirt','sandals','sweater','cardigan','raincoat','coat','gloves','jeans','sneakers','dress','suit','pants',
                'vest','blouse','jacket','sunglasses','bag','shoes']

In [8]:
# package import
from google_images_download import google_images_download 
from imageai.Prediction import ImagePrediction
from PIL import Image
from io import BytesIO
import csv
import random
import requests
import warnings
warnings.filterwarnings(action='ignore') 

ratings = pd.read_csv('item_rating.csv')

# item keyword 리스트
# 무작위 아이템아이디 생성
# 성별 입력 (남자 or 여자)
item_keyword = ['shorts','caps','T-shirt','sandals','sweater',
                'cardigan','raincoat','coat','gloves','jeans',
                'sneakers','dress','suit','pants','vest',
                'blouse','jacket','sunglasses','bag','shoes']
item_Id = random.randint(1,20)
gender = input('성별을 입력해주세요: ')

keywords = str("20대 " + gender + " " + item_keyword[(item_Id)-1] + " 추천")

response = google_images_download.googleimagesdownload()
arguments = {"keywords":keywords,"limit":1,"output_directory":'image',"no_download":True, "safe_search":True}
paths = response.download(arguments)

while True:
    itemscore = int(input('점수를 평가해주세요(1~5점): '))
    if 1 <= itemscore and itemscore <= 5:
        break
    else:
        print('warning: 1점에서 5점사이의 값을 입력하세요')

# 새로운 user row 추가를 위한 속성변수 선언         
new_userId = int(ratings.loc[len(ratings.index) - 1]['userId']) + 1
if gender == '남자':
    new_gender = 1
else: new_gender = 2 
new_itemId = item_Id
new_rating = itemscore

# row 추가
ratings.loc[len(ratings.index)] = [new_userId, new_gender, new_itemId, new_rating]

# csv파일에 저장하기전에 index 초기화 
# csv파일에 추가된 row저장 
# index 재설정 
ratings = ratings.set_index("userId")
ratings.to_csv('item_rating.csv')
ratings = pd.read_csv('item_rating.csv')

# url 파싱 
url = paths[keywords][0]
resp = requests.get(url)
img = Image.open(BytesIO(resp.content))


성별을 입력해주세요: 여자

Item no.: 1 --> Item name = 20\ub300 \uc5ec\uc790 sandals \ucd94\ucc9c
Evaluating...
Starting to Print Image URLS
Image URL: https://post-phinf.pstatic.net/MjAxNzA1MDhfMjgg/MDAxNDk0MjI2ODM2NjA2.3x3u-WOn7QKHyycChFDc4SP7Z05eB25M20y-fYaO7IQg.63X43gSZCm9SbdoadcYumWRGBhCi8WsEJqFaPoTB5dkg.JPEG/2.jpg?type=w1200
Printed url without downloading

Errors: 0

점수를 평가해주세요(1~5점): 2


In [11]:
item_keyword[0]

'shorts'

In [12]:
item_id = random.randint(1,20)

In [13]:
item_id

12

In [14]:
type(item_id)

int

In [15]:
keywords

'20대 여자 sandals 추천'

In [17]:
ratings.tail()

AttributeError: 'function' object has no attribute 'tail'

In [ ]:
ratings[0:1]

In [ ]:
new_userId = int(ratings.loc[len(ratings.index) - 1]['userId']) + 1

if gender == '남자':
    new_gender = 1
else: new_gender = 2 

new_itemId = item_Id
new_rating = itemscore

ratings.loc[len(ratings.index)] = [new_userId, new_gender, new_itemId, new_rating]

In [ ]:
ratings.tail(2)

In [ ]:
new_userId = int(ratings.loc[len(ratings.index)-1]['userId']) + 1

In [ ]:
new_userId

In [ ]:
ratings.tail(2)

In [ ]:
len(ratings.index)

In [10]:
ratings.head(5)

AttributeError: 'function' object has no attribute 'head'

In [20]:
ratings

,userId,gender,itemId,rating
0,1,2,16,3
1,1,2,11,1
2,1,2,8,1
3,1,2,15,1
4,1,2,19,5
5,1,2,20,3
6,1,2,12,4
7,1,2,4,2
8,1,2,13,4
9,1,2,1,4
